In [13]:
!pip install transformers --upgrade

     |████████████████████████████████| 675kB 4.7MB/s 
     |████████████████████████████████| 890kB 18.5MB/s 
     |████████████████████████████████| 1.1MB 27.8MB/s 
     |████████████████████████████████| 3.8MB 19.8MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=d7e761f74e50f075654d379cfa79a66e997e347e595092fe4de4497ed49cf4fa
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [15]:
# Load SQuAD Dataset
from transformers.data.processors.squad import SquadResult, SquadV1Processor, SquadV2Processor, squad_convert_examples_to_features
import tensorflow_datasets as tfds
tfds_examples = tfds.load("squad")
examples = SquadV1Processor().get_examples_from_dataset(tfds_examples, evaluate=True)
# examples = SquadV1Processor().get_examples_from_dataset(tfds_examples, evaluate=False)

INFO:absl:No config specified, defaulting to first: squad/plain_text
INFO:absl:Load pre-computed datasetinfo (eg: splits) from bucket.
INFO:absl:Loading info from GCS for squad/plain_text/1.0.0
INFO:absl:Generating dataset squad (/root/tensorflow_datasets/squad/plain_text/1.0.0)


INFO:absl:URL https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v1.1.json already downloaded: reusing /root/tensorflow_datasets/downloads/rajpurkar_SQuAD-explorer_train-v1.1NSdmOYa4KVr09_zf8bof8_ctB9YaIPSHyyOKbvkv2VU.json.
INFO:absl:Skipping extraction for /root/tensorflow_datasets/downloads/rajpurkar_SQuAD-explorer_train-v1.1NSdmOYa4KVr09_zf8bof8_ctB9YaIPSHyyOKbvkv2VU.json (method=NO_EXTRACT).
INFO:absl:URL https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v1.1.json already downloaded: reusing /root/tensorflow_datasets/downloads/rajpurkar_SQuAD-explorer_dev-v1.1lapqUtXWpzVWM2Z1PKUEkqZYAx2nTzAaxSOLA5Zpcsk.json.
INFO:absl:Skipping extraction for /root/tensorflow_datasets/downloads/rajpurkar_SQuAD-explorer_dev-v1.1lapqUtXWpzVWM2Z1PKUEkqZYAx2nTzAaxSOLA5Zpcsk.json (method=NO_EXTRACT).
INFO:absl:Generating split train


INFO:absl:generating examples from = /root/tensorflow_datasets/downloads/rajpurkar_SQuAD-explorer_train-v1.1NSdmOYa4KVr09_zf8bof8_ctB9YaIPSHyyOKbvkv2VU.json


Shuffling and writing examples to /root/tensorflow_datasets/squad/plain_text/1.0.0.incomplete4P3KS1/squad-train.tfrecord


INFO:absl:Done writing /root/tensorflow_datasets/squad/plain_text/1.0.0.incomplete4P3KS1/squad-train.tfrecord. Shard lengths: [87599]
INFO:absl:Generating split validation


INFO:absl:generating examples from = /root/tensorflow_datasets/downloads/rajpurkar_SQuAD-explorer_dev-v1.1lapqUtXWpzVWM2Z1PKUEkqZYAx2nTzAaxSOLA5Zpcsk.json


Shuffling and writing examples to /root/tensorflow_datasets/squad/plain_text/1.0.0.incomplete4P3KS1/squad-validation.tfrecord


INFO:absl:Done writing /root/tensorflow_datasets/squad/plain_text/1.0.0.incomplete4P3KS1/squad-validation.tfrecord. Shard lengths: [10570]
INFO:absl:Skipping computing stats for mode ComputeStatsMode.AUTO.
INFO:absl:Constructing tf.data.Dataset for split None, from /root/tensorflow_datasets/squad/plain_text/1.0.0


Dataset squad downloaded and prepared to /root/tensorflow_datasets/squad/plain_text/1.0.0. Subsequent calls will reuse this data.


10570it [00:13, 803.90it/s]


In [16]:
from transformers import BertTokenizer, BertModel
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

features, raw_dataset = squad_convert_examples_to_features(
            examples=examples,
            tokenizer=tokenizer,
            max_seq_length=512,
            doc_stride=128,
            max_query_length=64,
            is_training=False,
            return_dataset="pt",
            #threads=args.threads,
)


add example index and unique id: 100%|██████████| 10570/10570 [00:00<00:00, 729294.18it/s]


In [ ]:
print(len(raw_dataset))

87748


In [40]:
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
import numpy as np
import nltk
import itertools
nltk.download('stopwords')
nltk.download('punkt')
# randomly drop K stop words in sentence, return N variations
def dropwords(sentence, N, K = None):
    stop_words = set(stopwords.words('english')) 
    word_tokens = word_tokenize(sentence) 
    stop_words_indices = []
    for idx, word in enumerate(word_tokens):
        if word in stop_words:
            stop_words_indices.append(idx)
    dropped_sentences = []
    if len(stop_words_indices) == 0:
        return dropped_sentences
    
    # Randomly choose K indices from all stop_words_indices
    # If K is not specified, generate all possible combinations
    comb = []
    if not K:
        for r in range(1, len(stop_words_indices)+1):
            comb += list(itertools.combinations(stop_words_indices, r))
    else:
        K = min(len(stop_words_indices), max(1, K))
        comb += list(itertools.combinations(stop_words_indices, K))
    
    N_chosen_indices = [comb[idx] for idx in np.random.choice(len(comb), size = min(N, len(comb)), replace = False)]
    for chosen_indices in N_chosen_indices:
        new_words = [word_tokens[idx] for idx in range(len(word_tokens)) if idx not in chosen_indices]
        dropped_sentence = ' '.join(new_words)
        dropped_sentences.append(dropped_sentence)
    return dropped_sentences

raw_sentence = "Andy's friend just ate an apple and a banana?!"
dropped_sentences = dropwords(raw_sentence, N = 2, K = None)
print(raw_sentence)
print(dropped_sentences)
print(tokenizer.tokenize(raw_sentence))
for dropped_sentence in dropped_sentences:
    print(tokenizer.tokenize(dropped_sentence))

dropped_sentences = dropwords(raw_sentence, N = 2, K = 2)
print(raw_sentence)
print(dropped_sentences)
print(tokenizer.tokenize(raw_sentence))
for dropped_sentence in dropped_sentences:
    print(tokenizer.tokenize(dropped_sentence))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[3, 5, 7, 8]
Andy's friend just ate an apple and a banana?!
["Andy 's friend just ate apple and banana ? !", "Andy 's friend ate apple a banana ? !"]
['andy', "'", 's', 'friend', 'just', 'ate', 'an', 'apple', 'and', 'a', 'banana', '?', '!']
['andy', "'", 's', 'friend', 'just', 'ate', 'apple', 'and', 'banana', '?', '!']
['andy', "'", 's', 'friend', 'ate', 'apple', 'a', 'banana', '?', '!']
[3, 5, 7, 8]
Andy's friend just ate an apple and a banana?!
["Andy 's friend ate an apple and banana ? !", "Andy 's friend ate apple and a banana ? !"]
['andy', "'", 's', 'friend', 'just', 'ate', 'an', 'apple', 'and', 'a', 'banana', '?', '!']
['andy', "'", 's', 'friend', 'ate', 'an', 'apple', 'and', 'banana', '?', '!']
['andy', "'", 's', 'friend', 'ate', 'apple', 'and', 'a', '

In [21]:
import pickle
with open('SQuAD_v1.1_dev.pickle','rb') as f:
    score_dict = pickle.load(f)
print(len(score_dict))
print(len(raw_dataset))

10650
10650


In [54]:
import sys
from torch.utils.data import Dataset
import torch
from torch.utils.data import DataLoader
import math
from collections import Counter
class MyTensorDataset(Dataset):
    def __init__(self, raw_dataset, tokenizer, importance_score_dict, is_training = False, sample_ratio = 2.0, p_replace = 0.1, p_dropword = 0.1, p_misspelling = 0.1):
        # A list of tuples of tensors
        self.dataset = [] 
        aug_dataset = []
        raw_dataset = [raw_dataset.__getitem__(idx) for idx in range(len(raw_dataset))]
        
        # Normalize probabilities of each augmentation
        probs = [p_dropword, p_replace, p_misspelling]
        probs = [p/sum(probs) for p in probs]
        augmentation_types = {
            'drop': 'drop_stopword',
            'synonym': 'replace_synonym',
            'misspelling': 'replace_misspelling'
        } 
        num_examples = len(raw_dataset)
        num_aug_examples = math.ceil(num_examples * sample_ratio)
        orig_indices = list(range(num_examples))

        # Randomly sample indices of data in original dataset with replacement
        aug_indices = np.random.choice(orig_indices, size = num_aug_examples)
        aug_freqs = Counter(aug_indices)

        # Reamining number of each agumentation types after exhausting previous example's variations
        remaining_count = {}
        for aug_type in augmentation_types.keys():
            remaining_count[aug_type] = 0
        
        for aug_idx, count in aug_freqs.items():
            # Get frequency of each augmentation type for current example with replacement
            aug_type_sample = np.random.choice(list(augmentation_types.keys()), size = count, p = probs)
            aug_type_freq = Counter(aug_type_sample)
            for aug_type in aug_type_freq.keys():
                aug_type_freq[aug_type] += remaining_count[aug_type]
            
            # Get raw data from original dataset and get corresponding importance score
            raw_data = raw_dataset[aug_idx]
            if is_training:
                input_ids, attention_masks, token_type_ids, start_positions, end_positions, cls_index, p_mask, is_impossible = raw_data
            else:
                input_ids, attention_masks, token_type_ids, feature_index, cls_index, p_mask = raw_data
            seq_len = len(input_ids) # sequence length for padding
            tokens = tokenizer.convert_ids_to_tokens(input_ids)
            sep_id = [idx for idx,token in enumerate(tokens) if token=='[SEP]']
            question = tokenizer.convert_tokens_to_string(tokens[1:sep_id[0]])
            context = tokenizer.convert_tokens_to_string(tokens[sep_id[0]+1:sep_id[1]])
            print(question)
            print(context)
            importance_score = importance_score_dict[aug_idx]
            print(importance_score)
            print('+'*60)
            print(aug_type_freq)
            print(raw_data)
            print('+'*60)
            for aug_type, aug_times in aug_type_freq.items():
                aug_questions = []
                aug_questions = dropwords(question, N = 1, K = 1)
                # aug_questions = augmentation_types[aug_type](question, importance_score, 'topK')
                for aug_question in aug_questions:
                    data_dict = tokenizer.encode_plus(aug_question,context,
                                                      pad_to_max_length = True,
                                                      max_length = seq_len,
                                                      is_pretokenized = False,
                                                      return_token_type_ids = True,
                                                      return_attention_mask = True,
                                                      return_tensors = 'pt',
                                                    )
                    if is_training:
                        aug_dataset.append(tuple([data_dict['input_ids'][0],
                                                 data_dict['attention_mask'][0],
                                                 data_dict['token_type_ids'][0],
                                                 start_position,
                                                 end_position,
                                                 cls_index,
                                                 p_mask,
                                                 is_impossible,
                                                  ]))
                    else:
                        aug_dataset.append(tuple([data_dict['input_ids'][0],
                                                 data_dict['attention_mask'][0],
                                                 data_dict['token_type_ids'][0],
                                                 feature_index,
                                                 cls_index,
                                                 p_mask,
                                                  ]))
                remaining_count[aug_type] = aug_times - len(aug_questions)
                print(aug_dataset)
                raise NameError('HiThere')
        self.dataset = raw_dataset + aug_dataset

    def __getitem__(self, index):
        return self.dataset[index]

    def __len__(self):
        return len(self.dataset)

dataset = MyTensorDataset(raw_dataset, tokenizer, score_dict)


many organizations have recently called for broader acceptance of which community ?
nevertheless , the united methodist church " implore [ s ] families and churches not to reject or condemn lesbian and gay members and friends " and commits itself to be in ministry with all persons , affirming that god ' s grace , love , and forgiveness is available to all . additionally , many organizations , conferences , and congregations have recently called for broader acceptance of the lgbt community within the umc . for example , the connectional table , a governing committee , has voted in favor of a proposal that calls for a localized option , which would permit ministers to officiate same - sex weddings , and it would allow conferences to ordain gay clergy . moreover , many conferences have taken a position by voting in favor of same - gender marriages with resolutions .
[('many', 61.59324264526367), ('organizations', 54.85578918457031), ('have', 4.494980812072754), ('recently', 23.86591529846

NameError: ignored